### Custom Architecture

We have seen routing for a defined device, but we can also see how different connectivities perform by defining our own architectures. For this we need the `Architecture` class, which can be initialized with a list of pairs of qubit labels showing which qubits are connected. Make sure to only define each edge once (i.e. don't add (1, 2) and (2, 1)). Let's build the following example architecture, where lines show connections between qubit labels:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0<br>
&nbsp;&nbsp;/&nbsp;&nbsp;|&nbsp;&nbsp;\ <br>
1&nbsp;&nbsp;&nbsp;2&nbsp;&nbsp;&nbsp;3<br>
&nbsp;&nbsp;\ &nbsp;|&nbsp;&nbsp;/<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4

In [1]:
import cirq
from pytket import Architecture

number_nodes = 5
custom_arch = Architecture([(0, 1), (0, 2), (0, 3), (1, 4), (2, 4), (3, 4)], number_nodes)

Next, build a circuit. We will need the `IndexedQubit` type from `pytket` to do this in order to later use the custom architecture.

In [2]:
from pytket.qubits import IndexedQubit

qubits = [IndexedQubit(i) for i in range(5)]

simple_circuit = cirq.Circuit.from_ops(
        cirq.H(qubits[0]),
        cirq.X(qubits[1]),
        cirq.CNOT(qubits[0], qubits[4]),
        cirq.Y(qubits[2]),
        cirq.CNOT(qubits[2], qubits[3]),
        cirq.CNOT(qubits[1], qubits[3]),
        cirq.Y(qubits[3]),
        cirq.CNOT(qubits[3], qubits[4]),
        cirq.Z(qubits[4])
    )
simple_circuit

0: ───H───@───────────────────────────
          │
1: ───X───┼───────────@───────────────
          │           │
2: ───────┼───Y───@───┼───────────────
          │       │   │
3: ───────┼───────X───X───Y───@───────
          │                   │
4: ───────X───────────────────X───Z───

In [3]:
from pytket._route_wrapper import route_circuit_arc
route_circuit_arc(simple_circuit, custom_arch, qubits)

0: ───H───×───────@───×───@───────────────
          │       │   │   │
1: ───X───┼───────┼───×───┼───────────────
          │       │       │
2: ───Y───×───@───┼───────┼───────────────
              │   │       │
3: ───────────┼───X───────X───Y───@───────
              │                   │
4: ───────────X───────────────────X───Z───

Two swaps are added to make the interactions between qubits (0, 4), (2, 3) and (1, 3) possible.<br>
Try creating your own architectures and circuits and seeing how costly routing is.